In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from splinter import Browser
import warnings
import pgeocode
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


-----------

# Using pgeocode library

In [2]:
csvData = pd.read_csv('data/score_df.csv')
csvDataPC = csvData['postal_code']

In [3]:
coordinates = []
postalCodes = []
for i in csvDataPC:
    country_code = pgeocode.Nominatim('ca')
    coord = country_code.query_postal_code(str(i))
    x = [coord.latitude, coord.longitude]
    coordinates.append(x)
    postalCodes.append(str(i))

In [4]:
coordinatesDF = pd.DataFrame(coordinates, postalCodes)
coordinatesDF.reset_index(inplace=True)
coordinatesDF.columns=['postal_codes', 'lat', 'long']
coordinatesDF.head()

postal_codes      lat      long
0      T2V 1M4  50.9819 -114.1004
1      T2V 2G9  50.9819 -114.1004
2      T2T 4L7  51.0242 -114.1004
3      T2G 1M1  51.0272 -114.0349
4      T3H 5Y7  51.0419 -114.2000

In [5]:
coordinatesDF.to_csv('data/CalgaryCoordinatesDF.csv', index=False)

----------------

# Using Geocoder website

In [2]:
url = 'https://geocoder.ca/?locate='
postalCode = 'T2L+0C9'
page = requests.get(url+postalCode)
soup = BeautifulSoup(page.text, "html.parser")
#print(soup.prettify())

In [3]:
tables = pd.read_html(url+postalCode)
table_df = tables[1]
table_df.head()

0  \
0  Dissemination Area Data for T2L0C9 | Demograph...   

                                                   1  
0  51.091844, -114.126368 Neighborhood of Charles...

In [4]:
title = soup.find('li', class_='pull-right').get_text()
print(title)

 51.091844,-114.126368


In [21]:
csvData = pd.read_csv('data/score_df.csv')
csvData.head()

postal_code  walk_score  bike_score  transit_score
0     T2V 1M4        58.0        61.0           55.0
1     T2V 2G9        57.0        75.0           57.0
2     T2T 4L7        38.0        82.0           38.0
3     T2G 1M1        64.0        78.0           49.0
4     T3H 5Y7        65.0        89.0           40.0

In [23]:
csvDataPC = csvData['postal_code']
testCSV = csvDataPC.head(20)

csvDataPC = csvDataPC.str.replace(' ', '+')
testCSV = csvDataPC.head(20)

In [27]:
coordinates = []
postalCodes = []
for i in csvDataPC:
    url = 'https://geocoder.ca/?locate='
    page = requests.get(url+str(i))
    time.sleep(2)
    soup = BeautifulSoup(page.text, "html.parser")
    try:
        title = soup.find('li', class_='pull-right').get_text()
    except:
        title = 'None'
    coordinates.append(title)
    #postalCodesP = i.str.replace('+', ' ')
    postalCodes.append(i)

In [28]:
coordinatesDF = pd.DataFrame(coordinates, postalCodes)
coordinatesDF.reset_index(inplace=True)

In [29]:
coordinatesDF.columns=['postal_codes', 'coordinates']
coordinatesDF.head()

postal_codes             coordinates
0      T2V 1M4   50.967013,-114.084990
1      T2V 2G9   50.989524,-114.074253
2      T2T 4L7   51.013395,-114.101767
3      T2G 1M1   51.036103,-114.038056
4      T3H 5Y7   51.044200,-114.216072

In [30]:
coordinatesDF.to_csv('data/coordinatesDF.csv', index=False)